# Using Hand Crafted Features(a)

In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

In [2]:
# Paths to dataset
dataset_path = r"classification_dataset"  # Using raw strings bacause the file names contain backslashes and escape characters
mask_path = os.path.join(dataset_path, "with_mask")
no_mask_path = os.path.join(dataset_path, "without_mask")

In [3]:
#loading a sample image to show that images with 0_0_≈˙◊¢ are not loading correctly
sample_image = cv2.imread('classification_dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png')
if sample_image is None:
    print('Image not loaded correctly')


Image not loaded correctly


This shows that the images with this kind of a name are not being loaded.Manual inspection shows that these kind of images only exist in the with_mask category.

In [4]:
import os

# Define the folder path where images are stored
folder_path = mask_path

special_chars = "≈˙◊¢" # The characters to be replaced

for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)

    # Replace specific special characters with '_'
    new_filename = filename
    for char in special_chars:
        new_filename = new_filename.replace(char, "_")

    new_path = os.path.join(folder_path, new_filename)

    # Rename the file if necessary
    if old_path != new_path:
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")

print("Renaming completed!")



Renamed: 0_0_≈˙◊¢ 2020-02-23 132115.png → 0_0_____ 2020-02-23 132115.png
Renamed: 0_0_≈˙◊¢ 2020-02-23 132400.png → 0_0_____ 2020-02-23 132400.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 171804.png → 0_0_____ 2020-02-24 171804.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 172039.png → 0_0_____ 2020-02-24 172039.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 202509.png → 0_0_____ 2020-02-24 202509.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 205216.png → 0_0_____ 2020-02-24 205216.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 215234.png → 0_0_____ 2020-02-24 215234.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 215615.png → 0_0_____ 2020-02-24 215615.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 220536.png → 0_0_____ 2020-02-24 220536.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 222124.png → 0_0_____ 2020-02-24 222124.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 224833.png → 0_0_____ 2020-02-24 224833.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 225329.png → 0_0_____ 2020-02-24 225329.png
Renamed: 0_0_≈˙◊¢ 2020-02-24 225427.png → 0_0_____ 2020-02-24 225427.png
Renamed: 0_0_≈˙◊¢ 2020-02-25 150422.png → 0_0_____ 

Now there will be no problem in loading any of the images.

In [5]:
# Feature extraction functions
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                      block_norm='L2-Hys', visualize=True)
    return features

def extract_lbp_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [6]:
# Iterate directory
def count_images(dir_path):
    count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            count += 1
    return count
print('Images with Mask:',count_images(mask_path))
print('Images without Mask:',count_images(no_mask_path))


Images with Mask: 2165
Images without Mask: 1930


In [7]:
# Load dataset and extract features
X, y = [], []

for label, path in enumerate([mask_path, no_mask_path]):  # 0: with_mask, 1: without_mask
    for file in os.listdir(path):
        img_path = r"{}".format(os.path.join(path, file))  # Use raw string path
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, (128, 128))
            # Extract features
            hog_feat = extract_hog_features(image)
            lbp_feat = extract_lbp_features(image)
            color_feat = extract_color_histogram(image)
            # Combine features
            combined_features = np.hstack([hog_feat, lbp_feat, color_feat])
            X.append(combined_features)
            y.append(label)
        else:
            print(f"Error loading image: {img_path}")

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

Error loading image: classification_dataset\with_mask\0_0_œ¬‘ÿ.png


There is still one image that is not loaded correctly.We are skipping it for now.

In [44]:
# **Simplified Train-Validation-Test Split (70%-15%-15%)**
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [67]:
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)
svm_val_preds = svm.predict(X_val)
svm_f1 = f1_score(y_val, svm_val_preds)
print(classification_report(y_val, svm_val_preds))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       334
           1       0.93      0.93      0.93       280

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



In [68]:
# Train Neural Network Classifier
nn = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
nn.fit(X_train, y_train)
nn_val_preds = nn.predict(X_val)
nn_f1 = f1_score(y_val, nn_val_preds)
print(classification_report(y_val, nn_val_preds))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       334
           1       0.95      0.95      0.95       280

    accuracy                           0.95       614
   macro avg       0.95      0.95      0.95       614
weighted avg       0.95      0.95      0.95       614



In [69]:
# Train XGBoost Classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb.fit(X_train, y_train)
xgb_val_preds = xgb.predict(X_val)
xgb_f1 = f1_score(y_val, xgb_val_preds)
print(classification_report(y_val, xgb_val_preds))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       334
           1       0.94      0.91      0.92       280

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



In [70]:
# **Choose Best Model Based on F1-Score**
best_model_name, best_model, best_f1_score = max(
    zip(["SVM", "Neural Network", "XGBoost"], 
        [svm, nn, xgb], 
        [svm_f1, nn_f1, xgb_f1]), 
    key=lambda x: x[2]
)

# **Final Testing on the Best Model**
final_test_preds = best_model.predict(X_test)
final_test_f1 = f1_score(y_test, final_test_preds)
final_test_report = classification_report(y_test, final_test_preds)

# **Print Results**
print(f"Validation F1-Scores: SVM={svm_f1:.4f}, NN={nn_f1:.4f}, XGBoost={xgb_f1:.4f}")
print(f"Best Model: {best_model_name} with Validation F1-Score = {best_f1_score:.4f}")
print(f"Test F1-Score for Best Model ({best_model_name}): {final_test_f1:.4f}")
print(final_test_report)

Validation F1-Scores: SVM=0.9250, NN=0.9483, XGBoost=0.9236
Best Model: Neural Network with Validation F1-Score = 0.9483
Test F1-Score for Best Model (Neural Network): 0.9236
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       336
           1       0.94      0.91      0.92       279

    accuracy                           0.93       615
   macro avg       0.93      0.93      0.93       615
weighted avg       0.93      0.93      0.93       615



# Using CNN(Automatic Feature Learning) - b

In [72]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# # Load datasets
# dataset_path = r"classification_dataset"  # Using raw strings bacause the file names contain backslashes and escape characters

# batch_size = 32
# img_size = (128, 128)
# train_ds = image_dataset_from_directory(
#     dataset_path,
#     validation_split=0.3,  
#     subset="training",
#     seed=42,
#     image_size=img_size,
#     batch_size=batch_size
# )

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xac in position 81: invalid start byte

As we can see here there is some problem with the naming convention.So in order to mitigate this we have decided to rename all the image files.We will be assigning them names sequentially like image_1,image_2 etc

In [ ]:
def rename(dataset_path,string):
    # Get all image files
    image_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

    # Rename files sequentially
    for index, filename in enumerate(image_files, start=1):
        old_path = os.path.join(dataset_path, filename)

        # Extract the file extension (e.g., .jpg, .png)
        extension = os.path.splitext(filename)[1]  # Includes the dot

        # Generate new filename
        new_filename = f"image_{string}_{index}{extension}"
        new_path = os.path.join(dataset_path, new_filename)

        # Rename the file
        if os.path.exists(new_path):
            continue
        os.rename(old_path, new_path)
        # print(f"Renamed: {filename} → {new_filename}")

print(" All images renamed")

rename(r"classification_dataset/with_mask", 'with_mask')
rename(r"classification_dataset/without_mask", 'without_mask')



 All images renamed


In [74]:
# Load datasets
dataset_path = r"classification_dataset"  # Using raw strings bacause the file names contain backslashes and escape characters

batch_size = 32
img_size = (128, 128)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,  
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 4092 files belonging to 2 classes.
Using 2865 files for training.


In [75]:

val_test_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,  
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 4092 files belonging to 2 classes.
Using 1227 files for validation.


In [76]:
# **Further split val_test into validation (15%) and test (15%)**
val_size = int(0.5 * len(val_test_ds))  # 50% of remaining 30% goes to validation
val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

In [81]:
def get_data(batch_size):
    train_ds = image_dataset_from_directory(
        dataset_path,
        validation_split=0.3,  
        subset="training",
        seed=42,
        image_size=img_size,
        batch_size=batch_size
    )
    val_test_ds = image_dataset_from_directory(
        dataset_path,
        validation_split=0.3,  
        subset="validation",
        seed=42,
        image_size=img_size,
        batch_size=batch_size
    )
    val_size = int(0.5 * len(val_test_ds))  # 50% of remaining 30% goes to validation
    val_ds = val_test_ds.take(val_size)
    test_ds = val_test_ds.skip(val_size)
    return train_ds, val_ds, test_ds

In [ ]:
# For creating a CNN model we have used a standard architecture with 2*2 pooling and increasing number of filters in each layer.We have also the Relu activation function
def create_cnn(activation='relu', optimizer=Adam(learning_rate=0.001)):
    model = Sequential([
        Conv2D(32, (3,3), activation=activation, input_shape=(128,128,3)),
        BatchNormalization(),
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2,2)),

        Conv2D(128, (3,3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(128, activation=activation),
        Dense(1, activation='sigmoid')  # becoz Binary classification
    ])
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Hyperparameter Variations
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32]
optimizers = [Adam, RMSprop]
activations = ['relu', 'tanh']

# Training and Comparing Different CNNs
results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            for activation in activations:
                print(f"\nTraining CNN with: LR={lr}, Batch={batch_size}, Optimizer={optimizer.__name__}, Activation={activation}")

                # Create and train CNN
                model = create_cnn(activation=activation, optimizer=optimizer(learning_rate=lr))
                train_ds, val_ds, test_ds = get_data(batch_size)
                # **Enable Caching & Prefetching for Faster Performance**
                AUTOTUNE = tf.data.AUTOTUNE
                train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
                val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
                test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
                history = model.fit(train_ds, validation_data=val_ds, epochs=15, batch_size=batch_size, verbose=1)

                # Evaluate on test set
                y_true = []
                y_pred = []
                for images, labels in test_ds:
                    preds = model.predict(images) > 0.5
                    y_true.extend(labels.numpy())
                    y_pred.extend(preds.astype("int32").flatten())

                f1 = f1_score(y_true, y_pred)
                acc = accuracy_score(y_true, y_pred)

                # Store results
                results.append({
                    'Learning Rate': lr,
                    'Batch Size': batch_size,
                    'Optimizer': optimizer.__name__,
                    'Activation': activation,
                    'Test Accuracy': acc,
                    'Test F1-Score': f1
                })

# Find the Best Model Based on F1-Score
best_model = max(results, key=lambda x: x['Test F1-Score'])



Training CNN with: LR=0.01, Batch=16, Optimizer=Adam, Activation=relu


c:\micromamba\envs\ml\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 4092 files belonging to 2 classes.
Using 2865 files for training.
Found 4092 files belonging to 2 classes.
Using 1227 files for validation.
Epoch 1/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 36s 175ms/step - accuracy: 0.7676 - loss: 14.3821 - val_accuracy: 0.8849 - val_loss: 0.3876
Epoch 2/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.9169 - loss: 0.2618 - val_accuracy: 0.9046 - val_loss: 0.2379
Epoch 3/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 31s 174ms/step - accuracy: 0.9425 - loss: 0.1742 - val_accuracy: 0.7549 - val_loss: 0.6814
Epoch 4/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 177ms/step - accuracy: 0.9512 - loss: 0.1240 - val_accuracy: 0.8553 - val_loss: 0.3646
Epoch 5/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 177ms/step - accuracy: 0.9358 - loss: 0.1674 - val_accuracy: 0.8454 - val_loss: 0.5389
Epoch 6/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 0.9382 - loss: 0.1569 - val_accuracy: 0.8586 - val_loss: 0.5782
Epoch 7/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 

c:\micromamba\envs\ml\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 4092 files belonging to 2 classes.
Using 2865 files for training.
Found 4092 files belonging to 2 classes.
Using 1227 files for validation.
Epoch 1/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 37s 180ms/step - accuracy: 0.6954 - loss: 0.8253 - val_accuracy: 0.7336 - val_loss: 0.5595
Epoch 2/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 0.5520 - loss: 0.6831 - val_accuracy: 0.4605 - val_loss: 1.4613
Epoch 3/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 0.7235 - loss: 0.5890 - val_accuracy: 0.6562 - val_loss: 0.6621
Epoch 4/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 0.6769 - loss: 0.6253 - val_accuracy: 0.4638 - val_loss: 0.7698
Epoch 5/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 33s 183ms/step - accuracy: 0.5879 - loss: 0.6754 - val_accuracy: 0.5395 - val_loss: 1.0650
Epoch 6/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 33s 181ms/step - accuracy: 0.5178 - loss: 0.7396 - val_accuracy: 0.5395 - val_loss: 1.0221
Epoch 7/15
180/180 ━━━━━━━━━━━━━━━━━━━━ 33s 183ms/step - accuracy: 0

...

Found 4092 files belonging to 2 classes.
Using 2865 files for training.
Found 4092 files belonging to 2 classes.
Using 1227 files for validation.
Epoch 1/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 324ms/step - accuracy: 0.8728 - loss: 0.3341 - val_accuracy: 0.8783 - val_loss: 0.3080
Epoch 2/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 313ms/step - accuracy: 0.9411 - loss: 0.1327 - val_accuracy: 0.9391 - val_loss: 0.1616
Epoch 3/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 313ms/step - accuracy: 0.9705 - loss: 0.0841 - val_accuracy: 0.9408 - val_loss: 0.1627
Epoch 4/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 311ms/step - accuracy: 0.9886 - loss: 0.0443 - val_accuracy: 0.9490 - val_loss: 0.1286
Epoch 5/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 310ms/step - accuracy: 0.9971 - loss: 0.0258 - val_accuracy: 0.9572 - val_loss: 0.1221
Epoch 6/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 310ms/step - accuracy: 1.0000 - loss: 0.0138 - val_accuracy: 0.9622 - val_loss: 0.1123
Epoch 7/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 311ms/step - accuracy: 0.9999 - loss: 

c:\micromamba\envs\ml\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 4092 files belonging to 2 classes.
Using 2865 files for training.
Found 4092 files belonging to 2 classes.
Using 1227 files for validation.
Epoch 1/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 35s 344ms/step - accuracy: 0.8415 - loss: 0.6437 - val_accuracy: 0.7089 - val_loss: 0.5082
Epoch 2/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 340ms/step - accuracy: 0.9727 - loss: 0.0813 - val_accuracy: 0.9589 - val_loss: 0.1324
Epoch 3/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 343ms/step - accuracy: 0.9939 - loss: 0.0214 - val_accuracy: 0.9539 - val_loss: 0.1240
Epoch 4/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 344ms/step - accuracy: 0.9982 - loss: 0.0110 - val_accuracy: 0.9572 - val_loss: 0.1461
Epoch 5/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 344ms/step - accuracy: 0.9991 - loss: 0.0053 - val_accuracy: 0.9655 - val_loss: 0.1268
Epoch 6/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 338ms/step - accuracy: 1.0000 - loss: 4.4394e-04 - val_accuracy: 0.9605 - val_loss: 0.1581
Epoch 7/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 309ms/step - accuracy: 1.0000 - lo

c:\micromamba\envs\ml\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 4092 files belonging to 2 classes.
Using 2865 files for training.
Found 4092 files belonging to 2 classes.
Using 1227 files for validation.
Epoch 1/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 318ms/step - accuracy: 0.8067 - loss: 0.5994 - val_accuracy: 0.8569 - val_loss: 0.3333
Epoch 2/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 308ms/step - accuracy: 0.9422 - loss: 0.1516 - val_accuracy: 0.9523 - val_loss: 0.1278
Epoch 3/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 307ms/step - accuracy: 0.9716 - loss: 0.0786 - val_accuracy: 0.9572 - val_loss: 0.1193
Epoch 4/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 310ms/step - accuracy: 0.9917 - loss: 0.0423 - val_accuracy: 0.9655 - val_loss: 0.1039
Epoch 5/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 309ms/step - accuracy: 0.9946 - loss: 0.0206 - val_accuracy: 0.9720 - val_loss: 0.0834
Epoch 6/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 308ms/step - accuracy: 0.9995 - loss: 0.0107 - val_accuracy: 0.9572 - val_loss: 0.1257
Epoch 7/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 307ms/step - accuracy: 1.0000 - loss: 

In [85]:
import pandas as pd
results = pd.DataFrame(results)
results.sort_values(by='Test F1-Score', ascending=False, inplace=True)
print(results.head())

    Learning Rate  Batch Size Optimizer Activation  Test Accuracy  \
8          0.0010          16      Adam       relu       0.970921   
23         0.0001          32   RMSprop       tanh       0.966074   
22         0.0001          32   RMSprop       relu       0.966074   
12         0.0010          32      Adam       relu       0.959612   
14         0.0010          32   RMSprop       relu       0.959612   

    Test F1-Score  
8        0.970684  
23       0.965854  
22       0.965854  
12       0.959612  
14       0.958541  


In [86]:

# Print Best Model Configuration and Results
print("\n Best CNN Configuration:")
print(f"Learning Rate: {best_model['Learning Rate']}")
print(f"Batch Size: {best_model['Batch Size']}")
print(f"Optimizer: {best_model['Optimizer']}")
print(f"Activation: {best_model['Activation']}")
print(f"Test Accuracy: {best_model['Test Accuracy']:.4f}")
print(f"Test F1-Score: {best_model['Test F1-Score']:.4f}")


 Best CNN Configuration:
Learning Rate: 0.001
Batch Size: 16
Optimizer: Adam
Activation: relu
Test Accuracy: 0.9709
Test F1-Score: 0.9707


# Comparing the results

In [89]:
import pandas as pd

# Extract the best CNN results from the hyperparameter tuning process
best_cnn_results = {
    "Model": "Best CNN",
    "Test Accuracy": best_model["Test Accuracy"],  # Best CNN Accuracy
    "Test F1-Score": best_model["Test F1-Score"]  # Best CNN F1-Score
}

# Extract handcrafted feature-based models' best results
handcrafted_results = [
    {"Model": "SVM", "Test Accuracy": accuracy_score(y_test, svm.predict(X_test)), "Test F1-Score": f1_score(y_test, svm.predict(X_test))},
    {"Model": "XGBoost", "Test Accuracy": accuracy_score(y_test, xgb.predict(X_test)), "Test F1-Score": f1_score(y_test, xgb.predict(X_test))},
    {"Model": "Neural Network (MLP)", "Test Accuracy": accuracy_score(y_test, nn.predict(X_test)), "Test F1-Score": f1_score(y_test, nn.predict(X_test))}
]

# Convert to DataFrame
comparison_results = pd.DataFrame(handcrafted_results + [best_cnn_results])
print(comparison_results.sort_values(by="Test F1-Score", ascending=False))


                  Model  Test Accuracy  Test F1-Score
3              Best CNN       0.970921       0.970684
2  Neural Network (MLP)       0.931707       0.923636
1               XGBoost       0.918699       0.906367
0                   SVM       0.902439       0.891697


We can clearly see that Automatic Feature Learning and Extraction always performs better than Handcrafted Features